# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')

# Import API key
from api_keys import geoapify_key

# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,shizunai-furukawacho,42.3339,142.3669,0.65,65,100,14.44,JP,1738809928
1,1,sangmelima,2.9333,11.9833,20.34,98,98,1.68,CM,1738809929
2,2,amahai,-3.3333,128.9167,29.73,63,90,0.77,ID,1738809930
3,3,buckeye,33.3703,-112.5838,20.82,33,0,0.00,US,1738809931
4,4,waitangi,-43.9535,-176.5597,16.01,75,39,2.68,NZ,1738809932


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# Configure the map plot
# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City", 
)

# Display the map
hvplot.save(humidity_map, "output_data/humidity_map.html")
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
cities_narrow_lats_df = city_data_df[(city_data_df['Lat'] >= -19) & (city_data_df['Lat'] <= 40)]

# Drop any rows with null values
ideal_weather_city_df = cities_narrow_lats_df.dropna()

# Display sample data
ideal_weather_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,sangmelima,2.9333,11.9833,20.34,98,98,1.68,CM,1738809929
2,2,amahai,-3.3333,128.9167,29.73,63,90,0.77,ID,1738809930
3,3,buckeye,33.3703,-112.5838,20.82,33,0,0.00,US,1738809931
6,6,newman,37.3138,-121.0208,8.92,68,0,1.61,US,1738809934
7,7,kambove,-10.8764,26.5969,16.79,99,100,1.24,CD,1738809935
...,...,...,...,...,...,...,...,...,...,...
573,573,oyama,36.3000,139.8000,6.11,18,40,5.42,JP,1738810583
575,575,serenje,-13.2325,30.2352,17.25,99,100,1.34,ZM,1738810585
576,576,kaga bandoro,6.9896,19.1874,25.81,23,100,1.34,CF,1738810586
582,582,acapulco de juarez,16.8634,-99.8901,23.08,75,0,3.27,MX,1738810504


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_city_df[['City', 'Country','Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,sangmelima,CM,2.9333,11.9833,98,
2,amahai,ID,-3.3333,128.9167,63,
3,buckeye,US,33.3703,-112.5838,33,
6,newman,US,37.3138,-121.0208,68,
7,kambove,CD,-10.8764,26.5969,99,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
limit = 20
radius = 10000
params = {
    "limit":limit,
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
        
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
sangmelima - nearest hotel: Hôtel l'Afamba
amahai - nearest hotel: Hotel Irene
buckeye - nearest hotel: Days Inn
newman - nearest hotel: No hotel found
kambove - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
maracas - nearest hotel: Pousada Vale Aprazível
kuala belait - nearest hotel: Hotel Sentosa
howrah - nearest hotel: Sun India Guest House
afaahiti - nearest hotel: Omati Lodge
nova vicosa - nearest hotel: Pousada Pontal da Barra
paraiso - nearest hotel: Cabinas
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
miranda - nearest hotel: Encantos
prado - nearest hotel: Pousada Recanto do Prado
port isabel - nearest hotel: Queen Isabel Inn
sinabang - nearest hotel: No hotel found
kuala tungkal - nearest hotel: Hotel Masa Kini
burang - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
hawaiian paradise park - ne

In [ ]:
# Export the Hotel into a csv
hotel_df.to_csv("output_data/hotel.csv", index_label="City_ID")

In [ ]:
# Read saved data
hotel_df = pd.read_csv("output_data/hotel.csv", index_col="City_ID")

# Display sample data
hotel_df.head()

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Lat",
    scale = 1,
    color = "City", 
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hvplot.save(hotel_map, "output_data/hotel_map.html")
hotel_map